In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab
import time

# Pick an allocator config from this dictionary
allocators = {    
    "FIRSTFIT": {"noMetrics": "./Allocator/FirstFitAllocator.wasm", "metrics": "./Allocator/FirstFitAllocatorMetrics.wasm"},
    "BESTFIT": {"noMetrics": "./Allocator/BestFitAllocator.wasm", "metrics": "./Allocator/BestFitAllocatorMetrics.wasm"},
    "BUDDY": {"noMetrics": "./Allocator/BuddyAllocator.wasm", "metrics": "./Allocator/BuddyAllocatorMetrics.wasm"}
}

# Still need to compile the code with P0's compileString and save it to a file
# Allocator is a string in the allocators dictionary
def runFile(wasmfile, allocator):
    import pywasm
    plotPoints = {}
    # Defines for the environment
    def logdata(idx, val):
        print("Called with",idx,val)
        plotPoints[idx] = val
    if allocator not in allocators:
        print("That is not a real allocator")
        return {}
    allocator = allocators[allocator]
    # Load both VMs now 
    noMetricVM = pywasm.load(allocator["noMetrics"], {'env': {'logdata':logdata}})
    metricVM = pywasm.load(allocator["metrics"], {'env': {'logdata':logdata}})

    def mallocNM(bytes):
        x = noMetricVM.exec("malloc",[bytes])
        return x
    def freeNM(addy):
        noMetricVM.exec("free",[addy])
    def readAsIntNM(addy, offset):
        return noMetricVM.exec("readAsInt", [addy, offset])
    def writeAsIntNM(value, addy, offset):
        noMetricVM.exec("writeAsInt", [addy, offset, value])
    def writeAsIntArrayNM(addy, offset,value):
        noMetricVM.exec("writeAsInt", [addy, offset, value])

    def mallocM(bytes):
        x =  metricVM.exec("malloc",[bytes])
        return x
    def freeM(addy):
        metricVM.exec("free",[addy])
    def readAsIntM(addy, offset):
        x =  metricVM.exec("readAsInt", [addy, offset])
        return x
    def writeAsIntM(value, addy, offset):
        metricVM.exec("writeAsInt", [addy, offset, value])
    def writeAsIntArrayM(addy, offset, value):
        metricVM.exec("writeAsInt", [addy, offset, value])

    def write(x):
        print(x,end="")
    def writeln():
        print("")
    def read():
        return int(input("Enter number: "))

    print("Running without metrics...")
    cTime = time.time()
    vm1 = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read, 'malloc':mallocNM, 'free':freeNM, 'readAsInt':readAsIntNM, 'writeAsInt':writeAsIntNM, 'writeAsIntArray':writeAsIntArrayNM}})
    tTaken = time.time() - cTime
    print("Running with metrics...")
    plotPoints = {} #Just in case...
    print(plotPoints)
    vm2 = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read, 'malloc':mallocM, 'free':freeM, 'readAsInt':readAsIntM, 'writeAsInt':writeAsIntM, 'writeAsIntArray':writeAsIntArrayM}})
    plotPoints["time"] = tTaken
    # Clean the dictionary to be more readably
    if 1 not in plotPoints: plotPoints[1] = 0
    plotPoints["numMemoryExpansions"] = plotPoints[1]
    del plotPoints[1]
    if 2 not in plotPoints: plotPoints[2] = 32
    plotPoints["heapSize"] = plotPoints[2]
    del plotPoints[2]
    if 3 not in plotPoints: plotPoints[3] = 0
    plotPoints["avgFreeBytesAtExpansion"] = plotPoints[3]
    del plotPoints[3]
    return plotPoints

def runpywasm(wasmfile):
    import pywasm
    def write(i): print(i)
    def writeln(): print('\n')
    def read(): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

Importing Jupyter notebook from P0.ipynb
Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb


### Making this thing work:

1. Compile the allocator WATs to wasms 
2. Call compileString from original P0 to compile your P0 code to a WAT
3. wat2wasm that file 
4. Run it with runFile(wasm, allocator)

In [3]:
!wat2wasm ./Allocator/FirstFitAllocator.wat
!wat2wasm ./Allocator/FirstFitAllocatorMetrics.wat

#This should work on McMaster's jhub instance, but evidently, not mine...

'wat2wasm' is not recognized as an internal or external command,
operable program or batch file.
'wat2wasm' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
compileString("""
program p;
  var x: reference array [1..3] of integer;
  new(x);
  begin x[1] := 2; x[2] := 3; x[3] := x[1] + x[2]; write(x[3]) end
  free(x);
""","testing.wat", target='wat')


Importing Jupyter notebook from CGwat.ipynb
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 2)
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 3)
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev 

In [3]:
#Testing
# A general test case that will run quickly, to make sure the environment is set up properly
compileString("""
program p;
  var x: reference array [1..3] of integer;
  new(x);
  begin x[1] := 2; x[2] := 3; x[3] := x[1] + x[2]; write(x[3]) end
  free(x);
""","testing.wat", target='wat')

Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 2)
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 3)
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Var(name = , le

In [4]:
#Testing2
#the purpose of this test case is to make sure array allocation works as expected
compileString("""
program p;
  var x: reference array [1..2] of integer;
  var y: reference array [1..2] of integer;
  new(x);
  new(y);
  begin x[1] := 2; x[2] := 3; x[2] := x[1] ; write(x[2]) end
  begin y[1] := 0; y[2] := 4; end
  free(y);
  free(x);
""","testing2.wat", target='wat')

Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 2, base = <class 'ST.Int'>), elementSize = 4, size = 2, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 2)
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 2, base = <class 'ST.Int'>), elementSize = 4, size = 2, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 3)
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 2, base = <class 'ST.Int'>), elementSize = 4, size = 2, tp = <class 'ST.Int'>))
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 2, base = <class 'ST.Int'>), elementSize = 4, size = 2, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Ref(name = , lev = -1, tp = <class 'ST.Int'>)
Var(name = x, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 2, base = <class 'ST.Int'>), elementSize = 4, siz

In [23]:
#Testing3 
#the purpose of this test case is to see how the algorithms respond to JUST allocation, no freeing
compileString("""
program p;
  var y: integer;
  var x: reference integer;
  new(x);
  begin x := 0; y:= 0;
   while y < 1500 do
     begin y := y + 1; new(x); x:= y end
   end;
  end;
""","testing3.wat", target='wat')

Var(name = x, lev = 0, tp = ReferenceObj(Type(name = integer, val = <class 'ST.Int'>), elementSize = 4, size = 1, tp = <class 'ST.Int'>))
ASSIG Var(name = x, lev = 0, tp = ReferenceObj(Type(name = integer, val = <class 'ST.Int'>), elementSize = 4, size = 1, tp = <class 'ST.Int'>)) Const(name = , tp = <class 'ST.Int'>, val = 0)
Var(name = y, lev = 0, tp = <class 'ST.Int'>)
ASSIG Var(name = y, lev = 0, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 0)
Var(name = y, lev = 0, tp = <class 'ST.Int'>)
Var(name = y, lev = 0, tp = <class 'ST.Int'>)
Var(name = y, lev = 0, tp = <class 'ST.Int'>)
ASSIG Var(name = y, lev = 0, tp = <class 'ST.Int'>) Var(name = , lev = -1, tp = <class 'ST.Int'>)
Var(name = x, lev = 0, tp = ReferenceObj(Type(name = integer, val = <class 'ST.Int'>), elementSize = 4, size = 1, tp = <class 'ST.Int'>))
Var(name = y, lev = 0, tp = <class 'ST.Int'>)
ASSIG Var(name = x, lev = 0, tp = ReferenceObj(Type(name = integer, val = <class 'ST.Int'>), elementSize =

In [51]:
#Testing4
#The purpose of this test case is to intersperse frees and allocations
compileString("""
program p;
  var x,y,z: reference integer;
  var a1: reference array [1..3] of integer;
  var a2: reference array [1..5] of integer;
  new(x);
  new(a1);
  new(a2);
  begin a1[1] := 2; a1[2] := 3; a1[3] := 1  end
  begin a2[1] := 10; a2[2] := 1; a2[3] := 7 ; a2[4] := 8 ; a2[5] := 9   end
  new(z);
  free(a1);
  while y < 4000 do
     begin y := y + 1; free(x); new(a1); free(a2);new(y); free(z); free(a1); new(a1); a[1] := y; new(x); x:= y; free(a1); new(z) ; new(a2); new(y); new(a1) end
   end;
  y := 0;
  while y < 4000 do
     begin y := y + 1; new(x); free(a2); new(a1); new(z); new(a2); free(x); free(a1); new(x); new(a1); new(a2) end
   end;
""","testing4.wat", target='wat')

Var(name = a1, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 2)
Var(name = a1, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 3)
Var(name = a1, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 3, base = <class 'ST.Int'>), elementSize = 4, size = 3, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 1)


In [26]:
#Testing5
compileString("""
program p;
  var x,y,z: reference integer;
  var a1: reference array [1..8] of integer;
  var a2: reference array [1..16] of integer;
  var a3: reference array [1..36] of integer;
  new(x);
  new(a1);
  new(a2);
  begin a1[1] := 2; a1[2] := 3; a1[3] := 1  end
  begin a2[1] := 10; a2[2] := 1; a2[3] := 7 ; a2[4] := 8 ; a2[5] := 9   end
  new(z);
  free(a1);
  while y < 1000 do
     begin y := y + 1; new(a3); free(x); new(a1); free(a2);new(y); free(z); free(a1); new(a1); a[1] := y; new(x); x:= y; free(a1); new(z) ; new(a2); new(y); new(a1) end
   end;
  y := 0;
  while y < 1000 do
     begin y := y + 1; new(x); free(a2); new(a1); new(z); new(a2); free(x); new(a3); free(a1); new(x); new(a1); new(a2) end
   end;
""","testing5.wat", target='wat')

Var(name = a1, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 8, base = <class 'ST.Int'>), elementSize = 4, size = 8, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 2)
Var(name = a1, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 8, base = <class 'ST.Int'>), elementSize = 4, size = 8, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 3)
Var(name = a1, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 8, base = <class 'ST.Int'>), elementSize = 4, size = 8, tp = <class 'ST.Int'>))
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 1)


In [2]:
#Testing6
#As opposed to looping, this test case is for arrays of different sizes
compileString("""
program p;
  var x,y,z: reference integer;
  var a1: reference array [1..6] of integer;
  var a2: reference array [1..7] of integer;
  var a3: reference array [1..15] of integer;
  var a4: reference array [1..2] of integer;
  begin 
   new(a1);
   a1[1]:= 1;
   a1[5] := 5;
   new(x);
   new(y);
   x:= 1;
   y:= 5;
   new(a2);
   a2[1]:= 1;
   a2[7] := 5;
   new(a3);
   a3[1]:= 2;
   a3[15] := 5;
   free(a1);
   new(z);
   new(a4);
   free(y);
   new(a1);
   new(a1);
   new(a1);
   free(z);
   new(a1);
   new(a1);
   new(a1);
   new(a1)
  end
""","testing6.wat", target='wat')

Importing Jupyter notebook from CGwat.ipynb
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 6)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 7)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 15)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 2)
Var(name = a1, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 6, base = <class 'ST.Int'>), elementSize = 4, size = 6, tp = <class 'ST.Int'>))
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
ASSIG Ref(name = , lev = -1, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 1)
LoadItem called with Const(n

In [10]:
#Testing7
#this test case is for different sized arrays 
compileString("""
program p;
  var x,y,z: reference integer;
  var a1: reference array [1..6] of integer;
  var a2: reference array [1..7] of integer;
  var a3: reference array [1..15] of integer;
  var a4: reference array [1..2] of integer;
  var a5: reference array [1..20] of integer;
  begin 
   new(a1);
   a1[1]:= 1;
   a1[5] := 5;
   new(x);
   new(y);
   new(a5);
   x:= 1;
   y:= 5;
   new(a2);
   a2[1]:= 1;
   a2[7] := 5;
   new(a3);
   a3[1]:= 2;
   a3[15] := 5;
   free(a1);
   new(z);
   free(a5);
   new(a4);
   while y < 4000 do
     begin y := y + 1; new(a4) end
   end;
   new(a1);
   new(a1);
   new(a5);
   free(a1);
   new(a1);
   free(a5);
   free(z);
   new(a1);
   new(a1);
   new(z);
   new(a5);
   new(a1);
   new(a1)
  end
""","testing7.wat", target='wat')

Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 6)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 7)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 15)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 2)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 20)
Var(name = a1, lev = 0, tp = ReferenceObj(Array(lower = 1, length = 6, base = <class 'ST.Int'>), elementSize = 4, size = 6, tp = <class 'ST.Int'>))
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
ASSIG Ref(name = , lev = -1, tp = 

In [32]:
#Testing8
#aOOOefOOi
#this test is intended to show the benefit of best fit over first fit
#in practice, however it only serves to prove how similar they are
compileString("""
program p;
  var a,b,c,d,e,f,g,h,i : reference integer;
  var y : integer;
  var a1: reference array [1..4] of integer;
  begin 
   y := 0;
   while y < 100 do
       begin y := y + 1; new(a); new(b); new(c); new(d); new(e); new(f); new(g); new(h); new(i); free(b); free(c); free(d); free(g); free(h); new(c); new(d); new(a1) end
   end;
end
""","testing8.wat", target='wat')

Expression returning Const(name = , tp = <class 'ST.Int'>, val = 1)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 4)
Var(name = y, lev = 0, tp = <class 'ST.Int'>)
Expression returning Const(name = , tp = <class 'ST.Int'>, val = 0)
ASSIG Var(name = y, lev = 0, tp = <class 'ST.Int'>) Const(name = , tp = <class 'ST.Int'>, val = 0)
LoadItem called with Const(name = , tp = <class 'ST.Int'>, val = 0)
Var(name = y, lev = 0, tp = <class 'ST.Int'>)
LoadItem called with Var(name = y, lev = 0, tp = <class 'ST.Int'>)
LoadItem called with Const(name = , tp = <class 'ST.Int'>, val = 100)
Expression returning Var(name = , lev = -1, tp = <class 'ST.Bool'>)
GenDo called with Var(name = , lev = -1, tp = <class 'ST.Bool'>)
LoadItem called with Var(name = , lev = -1, tp = <class 'ST.Bool'>)
Var(name = y, lev = 0, tp = <class 'ST.Int'>)
Var(name = y, lev = 0, tp = <class 'ST.Int'>)
LoadItem called with Var(name = y, lev = 0, tp = <class 'ST.Int'>)
LoadItem called with Const(name = , tp =

In [5]:
runFile("testing.wasm","FIRSTFIT")

NameError: name 'runFile' is not defined

In [10]:
runFile("testing.wasm","BESTFIT")

Running without metrics...
5Running with metrics...
{}
5

{'time': 0.0020341873168945312,
 'numMemoryExpansions': 0,
 'heapSize': 32,
 'avgFreeBytesAtExpansion': 0}

In [11]:
runFile("testing.wasm","BUDDY")

Running without metrics...
5Running with metrics...
{}
5

{'time': 0.004040956497192383,
 'numMemoryExpansions': 0,
 'heapSize': 32,
 'avgFreeBytesAtExpansion': 0}

In [12]:
runFile("testing2.wasm","FIRSTFIT")

Running without metrics...
2Running with metrics...
{}
2

{'time': 0.002992868423461914,
 'numMemoryExpansions': 0,
 'heapSize': 32,
 'avgFreeBytesAtExpansion': 0}

In [13]:
runFile("testing2.wasm","BESTFIT")

Running without metrics...
2Running with metrics...
{}
2

{'time': 0.003021717071533203,
 'numMemoryExpansions': 0,
 'heapSize': 32,
 'avgFreeBytesAtExpansion': 0}

In [14]:
runFile("testing2.wasm","BUDDY")

Running without metrics...
2Running with metrics...
{}
2

{'time': 0.005982398986816406,
 'numMemoryExpansions': 0,
 'heapSize': 32,
 'avgFreeBytesAtExpansion': 0}

In [15]:
runFile("testing3.wasm","FIRSTFIT")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 8
Called with 1 2
Called with 2 128
Called with 3 8
Called with 1 3
Called with 2 256
Called with 3 8
Called with 1 4
Called with 2 512
Called with 3 8
Called with 1 5
Called with 2 1024
Called with 3 8
Called with 1 6
Called with 2 2048
Called with 3 8
Called with 1 7
Called with 2 4096
Called with 3 8
Called with 1 8
Called with 2 8192
Called with 3 8
Called with 1 9
Called with 2 16384
Called with 3 8


{'time': 134.79944729804993,
 'numMemoryExpansions': 9,
 'heapSize': 16384,
 'avgFreeBytesAtExpansion': 8}

In [16]:
runFile("testing3.wasm","BESTFIT")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 8
Called with 1 2
Called with 2 128
Called with 3 8
Called with 1 3
Called with 2 256
Called with 3 8
Called with 1 4
Called with 2 512
Called with 3 8
Called with 1 5
Called with 2 1024
Called with 3 8
Called with 1 6
Called with 2 2048
Called with 3 8
Called with 1 7
Called with 2 4096
Called with 3 8
Called with 1 8
Called with 2 8192
Called with 3 8
Called with 1 9
Called with 2 16384
Called with 3 8


{'time': 164.25622534751892,
 'numMemoryExpansions': 9,
 'heapSize': 16384,
 'avgFreeBytesAtExpansion': 8}

In [17]:
runFile("testing3.wasm","BUDDY")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 0
Called with 1 2
Called with 2 128
Called with 3 0
Called with 1 3
Called with 2 256
Called with 3 0
Called with 1 4
Called with 2 512
Called with 3 0
Called with 1 5
Called with 2 1024
Called with 3 0
Called with 1 6
Called with 2 2048
Called with 3 0
Called with 1 7
Called with 2 4096
Called with 3 0
Called with 1 8
Called with 2 8192
Called with 3 0


{'time': 417.40708351135254,
 'numMemoryExpansions': 8,
 'heapSize': 8192,
 'avgFreeBytesAtExpansion': 0}

In [5]:
runFile("testing4.wasm","FIRSTFIT")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 8


{'time': 0.0029931068420410156,
 'numMemoryExpansions': 1,
 'heapSize': 64,
 'avgFreeBytesAtExpansion': 8}

In [6]:
runFile("testing4.wasm","BESTFIT")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 8


{'time': 0.003992795944213867,
 'numMemoryExpansions': 1,
 'heapSize': 64,
 'avgFreeBytesAtExpansion': 8}

In [7]:
runFile("testing4.wasm","BUDDY")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 16


{'time': 0.006994724273681641,
 'numMemoryExpansions': 1,
 'heapSize': 64,
 'avgFreeBytesAtExpansion': 16}

In [8]:
runFile("testing5.wasm","FIRSTFIT")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 24
Called with 1 2
Called with 2 128
Called with 3 22


{'time': 0.003988981246948242,
 'numMemoryExpansions': 2,
 'heapSize': 128,
 'avgFreeBytesAtExpansion': 22}

In [9]:
runFile("testing5.wasm","BESTFIT")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 24
Called with 1 2
Called with 2 128
Called with 3 22


{'time': 0.00302886962890625,
 'numMemoryExpansions': 2,
 'heapSize': 128,
 'avgFreeBytesAtExpansion': 22}

In [10]:
runFile("testing5.wasm","BUDDY")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 28
Called with 1 2
Called with 2 128
Called with 3 28


{'time': 0.00897073745727539,
 'numMemoryExpansions': 2,
 'heapSize': 128,
 'avgFreeBytesAtExpansion': 28}

In [2]:
runFile("testing6.wasm","FIRSTFIT")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 4
Called with 1 2
Called with 2 128
Called with 3 12
Called with 1 3
Called with 2 256
Called with 3 25
Called with 1 4
Called with 2 512
Called with 3 26


{'time': 0.017984867095947266,
 'numMemoryExpansions': 4,
 'heapSize': 512,
 'avgFreeBytesAtExpansion': 26}

In [3]:
runFile("testing6.wasm","BESTFIT")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 4
Called with 1 2
Called with 2 128
Called with 3 12
Called with 1 3
Called with 2 256
Called with 3 25
Called with 1 4
Called with 2 512
Called with 3 26


{'time': 0.01898479461669922,
 'numMemoryExpansions': 4,
 'heapSize': 512,
 'avgFreeBytesAtExpansion': 26}

In [4]:
runFile("testing6.wasm","BUDDY")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 8
Called with 1 2
Called with 2 128
Called with 3 20
Called with 1 3
Called with 2 256
Called with 3 36
Called with 1 4
Called with 2 512
Called with 3 48


{'time': 0.050899505615234375,
 'numMemoryExpansions': 4,
 'heapSize': 512,
 'avgFreeBytesAtExpansion': 48}

In [2]:
runFile("testing7.wasm","FIRSTFIT")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 4
Called with 1 2
Called with 1 3
Called with 2 256
Called with 3 8
Called with 1 4
Called with 2 512
Called with 3 10
Called with 1 5
Called with 2 1024
Called with 3 12
Called with 1 6
Called with 2 2048
Called with 3 12
Called with 1 7
Called with 2 4096
Called with 3 13
Called with 1 8
Called with 2 8192
Called with 3 14
Called with 1 9
Called with 2 16384
Called with 3 14


{'time': 60.10044741630554,
 'numMemoryExpansions': 9,
 'heapSize': 16384,
 'avgFreeBytesAtExpansion': 14}

In [3]:
runFile("testing7.wasm","BESTFIT")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 4
Called with 1 2
Called with 1 3
Called with 2 256
Called with 3 8
Called with 1 4
Called with 2 512
Called with 3 10
Called with 1 5
Called with 2 1024
Called with 3 12
Called with 1 6
Called with 2 2048
Called with 3 12
Called with 1 7
Called with 2 4096
Called with 3 13
Called with 1 8
Called with 2 8192
Called with 3 14
Called with 1 9
Called with 2 16384
Called with 3 14


{'time': 70.12833952903748,
 'numMemoryExpansions': 9,
 'heapSize': 16384,
 'avgFreeBytesAtExpansion': 14}

In [4]:
runFile("testing7.wasm","BUDDY")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 8
Called with 1 2
Called with 2 128
Called with 3 20
Called with 1 3
Called with 2 256
Called with 3 45
Called with 1 4
Called with 2 512
Called with 3 63
Called with 1 5
Called with 2 1024
Called with 3 52
Called with 1 6
Called with 2 2048
Called with 3 46
Called with 1 7
Called with 2 4096
Called with 3 41
Called with 1 8
Called with 2 8192
Called with 3 37


{'time': 197.40409326553345,
 'numMemoryExpansions': 8,
 'heapSize': 8192,
 'avgFreeBytesAtExpansion': 37}

In [28]:
runFile("testing8.wasm","FIRSTFIT")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 8
Called with 1 2
Called with 2 128
Called with 3 8
Called with 1 3
Called with 2 256
Called with 3 6
Called with 1 4
Called with 2 512
Called with 3 6
Called with 1 5
Called with 2 1024
Called with 3 5
Called with 1 6
Called with 2 2048
Called with 3 6
Called with 1 7
Called with 2 4096
Called with 3 9
Called with 1 8
Called with 2 8192
Called with 3 12


{'time': 165.1778118610382,
 'numMemoryExpansions': 8,
 'heapSize': 8192,
 'avgFreeBytesAtExpansion': 12}

In [30]:
runFile("testing8.wasm","BESTFIT")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 8
Called with 1 2
Called with 2 128
Called with 3 8
Called with 1 3
Called with 2 256
Called with 3 9
Called with 1 4
Called with 2 512
Called with 3 12
Called with 1 5
Called with 2 1024
Called with 3 16
Called with 1 6
Called with 2 2048
Called with 3 24
Called with 1 7
Called with 2 4096
Called with 3 38
Called with 1 8
Called with 2 8192
Called with 3 62


{'time': 159.10561609268188,
 'numMemoryExpansions': 8,
 'heapSize': 8192,
 'avgFreeBytesAtExpansion': 62}

In [31]:
runFile("testing8.wasm","BUDDY")

Running without metrics...
Running with metrics...
{}
Called with 1 1
Called with 2 64
Called with 3 0
Called with 1 2
Called with 2 128
Called with 3 0
Called with 1 3
Called with 2 256
Called with 3 8
Called with 1 4
Called with 2 512
Called with 3 6
Called with 1 5
Called with 2 1024
Called with 3 4
Called with 1 6
Called with 2 2048
Called with 3 4
Called with 1 7
Called with 2 4096
Called with 3 6


{'time': 368.2391891479492,
 'numMemoryExpansions': 7,
 'heapSize': 4096,
 'avgFreeBytesAtExpansion': 6}